In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
import math
#from rateFunction import *
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
def sortBuzzWords(buzzwords, index_value):
    return sorted(buzzwords, key=lambda x: x[index_value], reverse=True)

In [3]:
#df = spark.read.csv('../../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [4]:
#xay.csv = 330 MB
#Alla andra 1.1GB = 1024MB = 1024*1024KB = 1024*1024*1024B
p = '../../data/all-data/'
#EVERYTHING "25.5GB= 25.5 * 1000MB"
df = spark.read.csv([p+'xaa.csv',p+'xac.csv',p+'xae.csv',p+'xag.csv',p+'xai.csv',p+'xak.csv',p+'xam.csv',p+'xao.csv',p+'xaq.csv',
                     p+'xas.csv',p+'xau.csv',p+'xaw.csv',p+'xab.csv',p+'xad.csv',p+'xaf.csv',p+'xah.csv',p+'xaj.csv',p+'xal.csv',
                     p+'xan.csv',p+'xap.csv',p+'xar.csv',p+'xat.csv',p+'xav.csv',p+'xax.csv',p+'xay.csv'], sep="\t", header=False)



In [5]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [6]:
totalCountsDf = spark.read.csv(p+'googlebooks-eng-all-totalcounts-20090715.txt', sep="\t", header=False)

In [7]:
totalCountsDf = totalCountsDf.withColumnRenamed('_c0', 'year')
totalCountsDf = totalCountsDf.withColumnRenamed('_c1', 'match_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c2', 'page_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c3', 'volume_count')

In [8]:
totalCounts = sc.broadcast(totalCountsDf.collect())

In [9]:
totalCountsRDD = totalCountsDf.rdd.map(list)

In [10]:
totalCountsRDD.collect()

[['1520', '51191', '112', '1'],
 ['1527', '4384', '18', '1'],
 ['1541', '5056', '27', '1'],
 ['1574', '60089', '345', '1'],
 ['1575', '374033', '1059', '2'],
 ['1576', '26278', '81', '1'],
 ['1581', '32983', '107', '1'],
 ['1584', '382341', '926', '1'],
 ['1586', '199627', '512', '1'],
 ['1588', '24106', '60', '1'],
 ['1592', '253381', '816', '6'],
 ['1593', '13785', '80', '1'],
 ['1596', '7396', '57', '1'],
 ['1602', '3389', '26', '1'],
 ['1603', '178263', '872', '1'],
 ['1604', '5257', '31', '1'],
 ['1608', '4387', '26', '1'],
 ['1611', '45897', '226', '1'],
 ['1612', '39674', '107', '2'],
 ['1613', '10438', '24', '1'],
 ['1614', '13918', '120', '1'],
 ['1618', '57343', '157', '2'],
 ['1620', '82498', '334', '2'],
 ['1621', '6186', '27', '1'],
 ['1627', '9062', '36', '1'],
 ['1630', '73194', '309', '1'],
 ['1631', '464439', '952', '1'],
 ['1632', '42700', '149', '1'],
 ['1634', '94412', '261', '2'],
 ['1635', '204307', '527', '2'],
 ['1637', '383892', '527', '1'],
 ['1638', '118459',

In [11]:
totalCounts = sc.broadcast(dict(totalCountsRDD.map(lambda x: (x[0], (x[1],x[2],x[3]))).collect()))

In [12]:
totalCounts.value["1944"]

('709200138', '1865154', '7201')

In [13]:
totalCounts.value

{'1520': ('51191', '112', '1'),
 '1527': ('4384', '18', '1'),
 '1541': ('5056', '27', '1'),
 '1574': ('60089', '345', '1'),
 '1575': ('374033', '1059', '2'),
 '1576': ('26278', '81', '1'),
 '1581': ('32983', '107', '1'),
 '1584': ('382341', '926', '1'),
 '1586': ('199627', '512', '1'),
 '1588': ('24106', '60', '1'),
 '1592': ('253381', '816', '6'),
 '1593': ('13785', '80', '1'),
 '1596': ('7396', '57', '1'),
 '1602': ('3389', '26', '1'),
 '1603': ('178263', '872', '1'),
 '1604': ('5257', '31', '1'),
 '1608': ('4387', '26', '1'),
 '1611': ('45897', '226', '1'),
 '1612': ('39674', '107', '2'),
 '1613': ('10438', '24', '1'),
 '1614': ('13918', '120', '1'),
 '1618': ('57343', '157', '2'),
 '1620': ('82498', '334', '2'),
 '1621': ('6186', '27', '1'),
 '1627': ('9062', '36', '1'),
 '1630': ('73194', '309', '1'),
 '1631': ('464439', '952', '1'),
 '1632': ('42700', '149', '1'),
 '1634': ('94412', '261', '2'),
 '1635': ('204307', '527', '2'),
 '1637': ('383892', '527', '1'),
 '1638': ('118459',

In [14]:
#df.count()

In [15]:
rdd = df.rdd.map(list)

In [16]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [17]:
#rdd.take(10)

In [18]:
#rdd2 = sc.parallelize(rdd.take(1000000))

In [19]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd2 = rdd.reduceByKey(groupData)

In [20]:
def yearRate(word):
    if len(word) == 0:
        return [('0',0,0.0,0,0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0,0,0,0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        try:
            yearPair = str(word[i][0])+"-"+str(word[i+1][0])
            diff = int(word[i+1][1])-int(word[i][1])
            diffProcent = (float(word[i+1][1])/float(word[i][1]))*100
            ratio=diffProcent/100
            buzzWordValue = (float(diff)/math.sqrt(float(totalCounts.value[word[i+1][0]][0])))*ratio
            #buzzWordValue = diff*diffProcent
            wordArray.append((yearPair,diff,diffProcent,int(word[i][1]),int(word[i+1][1]),buzzWordValue))
        except:
            return [('0',0,0.0,0,0,0.0)]
    return wordArray

In [21]:
def yearRateThresh(word,t):
    if len(word) == 0:
        return [('0',0,0.0,0,0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0,0,0,0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        try:
            if int(word[i][0])>t:
                yearPair = str(word[i][0])+"-"+str(word[i+1][0])
                diff = int(word[i+1][1])-int(word[i][1])
                diffProcent = (float(word[i+1][1])/float(word[i][1]))*100
                ratio=diffProcent/100
                buzzWordValue = (float(diff)/math.sqrt(float(totalCounts.value[word[i+1][0]][0])))*ratio
                #buzzWordValue = diff*diffProcent
                wordArray.append((yearPair,diff,diffProcent,int(word[i][1]),int(word[i+1][1]),buzzWordValue))
            else: 
                wordArray.append('0',0,0.0,0,0,0.0)
        except:
            return [('0',0,0.0,0,0,0.0)]
    return wordArray

In [22]:
#rdd3 = rdd2.map(lambda row: (row[0], yearRate(row[1])))

In [23]:
rdd4 = rdd2.map(lambda row: (row[0], yearRateThresh(row[1],1980)))

In [24]:
#rdd_count = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 1)))

In [25]:
#rdd_percent = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 2)))

In [26]:
#rdd_buzzword = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 5)))
rdd_buzzword = rdd4.map(lambda row: (row[0], sortBuzzWords(row[1], 5)))

In [27]:
#rdd_count2 = rdd_count.sortBy(lambda row: row[1][0][1], ascending=False)

In [28]:
#rdd_percent2 = rdd_percent.sortBy(lambda row: row[1][0][2], ascending=False)

In [29]:
rdd_buzzword2 = rdd_buzzword.sortBy(lambda row: row[1][0][5], ascending=False)

In [30]:
#rdd_count2.take(10)

In [31]:
#rdd_percent2.take(10)

[('ODAG',
  [('2004-2005', 18199, 1820000.0, 1, 18200, 3110.7228946394243),
   ('1995-1999', 6669, 133480.0, 5, 6674, 107.20701294719618),
   ('1999-2000', 27044, 505.21426430925976, 6674, 33718, 1.5102814073672357),
   ('1908-1927', 2, 300.0, 1, 3, 0.00018642558520888766),
   ('1981-1983', 2, 300.0, 1, 3, 0.00010259297343670175),
   ('1991-1995', 3, 250.0, 2, 5, 0.00010195205667088251),
   ('1987-1991', 1, 200.0, 1, 2, 3.021395403979744e-05),
   ('2002-2003', 27281, 183.31602736379185, 32744, 60025, 0.477642883681806),
   ('2000-2001', 12319, 136.53538169523696, 33718, 46037, 0.1814967372621273),
   ('1937-1970', 0, 100.0, 2, 2, 0.0),
   ('1970-1979', 0, 100.0, 2, 2, 0.0),
   ('1985-1987', 0, 100.0, 1, 1, 0.0),
   ('2001-2002',
    -13293,
    71.12539913547799,
    46037,
    32744,
    -0.09601206575606819),
   ('1927-1937', -1, 66.66666666666666, 3, 2, -2.0893133252147534e-05),
   ('1894-1908', -1, 50.0, 2, 1, -1.3787988541089295e-05),
   ('1979-1981', -1, 50.0, 2, 1, -8.9327780185

In [35]:
results = rdd_buzzword2.take(1000)

In [38]:
top_results = []
for r in results:
    top_results.append((r[0], r[1][0]))

In [39]:
top_results

[('PICmicro', ('1999-2000', 8375, 837600.0, 1, 8376, 775.4125559136177)),
 ('globalsources',
  ('2002-2003', 14580, 91225.0, 16, 14596, 127.03224842148875)),
 ('Korrika', ('1991-1993', 2926, 292700.0, 1, 2927, 124.3566383788824)),
 ('Qwilleran', ('1984-1986', 2356, 235700.0, 1, 2357, 89.39854319550852)),
 ('Baguazhang', ('1992-1994', 2307, 230800.0, 1, 2308, 74.53234494337426)),
 ('NEWSNET', ('1983-1984', 2878, 144000.0, 2, 2880, 68.36008542282465)),
 ('InterDev', ('1993-1997', 2051, 205200.0, 1, 2052, 54.002440607194295)),
 ('DataWindow', ('1999-2000', 2182, 218300.0, 1, 2183, 52.652600608115165)),
 ('Zengbu', ('1989-1990', 1881, 188200.0, 1, 1882, 52.399082410517295)),
 ('Parent loans',
  ('2003-2004', 4108, 137033.3333333333, 3, 4111, 51.49687553977515)),
 ('TrPs', ('1991-1992', 8393, 40066.66666666667, 21, 8414, 48.95825968043259)),
 ('QuickBase', ('2006-2007', 2168, 216900.0, 1, 2169, 41.97208552849481)),
 ('Nepad', ('2001-2002', 1914, 191500.0, 1, 1915, 37.22106576915492)),
 ('Ho

In [ ]:
#rdd3.coalesce(1).saveAsTextFile("Words-with-year-list.txt")